## Final Training Data

Using the output of our Batch process from the previous notebook - we create our final training dataset

In [1]:
import os
import re
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
import os
import json
from typing import List
import os
import openai

from dotenv import load_dotenv
load_dotenv()

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph


os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY')
database = os.getenv('NEO4J_DATABASE')
uri, user, password = os.getenv('NEO4J_URI'), os.getenv('NEO4J_USER'),'@PromptEngg'
DEMO_URL = uri
DEMO_DATABASES = [database];

In [2]:
# Load the JSONL file into a DataFrame
df = pd.read_json("text2cypher_batch_output.jsonl", lines=True)
df

,id,custom_id,response,error
0,batch_req_7b4DtF3MyMdqzud1PSUnnIKS,request-0,"{'status_code': 200, 'request_id': '61c2704410...",NaN
1,batch_req_1y9s4768GYy9OhzsX1xBHBhv,request-1,"{'status_code': 200, 'request_id': 'cbf5dc7910...",NaN
2,batch_req_Xm8fx8BLHY1uCMeEoJSKepYv,request-2,"{'status_code': 200, 'request_id': 'd7f080255a...",NaN
3,batch_req_11qNOU3QfOXyjQRn1w67gxVG,request-3,"{'status_code': 200, 'request_id': '5388a58dc4...",NaN
4,batch_req_HZtYiQ6Cqx9zbE3g4r5pXfng,request-4,"{'status_code': 200, 'request_id': '90f1ed5ff8...",NaN
...,...,...,...,...
207,batch_req_49d9JjN7dudXWFBbRlZxhpRW,request-207,"{'status_code': 200, 'request_id': 'c6a7643900...",NaN
208,batch_req_cgqbvheptVS8TomCrln7o940,request-208,"{'status_code': 200, 'request_id': 'b5e5a7570f...",NaN
209,batch_req_uOl2Fg4mi5urKExf3fGQ78fp,request-209,"{'status_code': 200, 'request_id': 'f90276cda3...",NaN
210,batch_req_85ZVER9jjwOnuXdtKvWx4WuL,request-210,"{'status_code': 200, 'request_id': '26bf22c2f7...",NaN


In [3]:
def extract_text(text):
    # Adjust pattern to capture after ```cypher and spans multiple lines until ```
    pattern = r"```cypher\n(.*?)\n```"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        # Return the extracted text if triple backticks are present
        return match.group(1).strip()
    else:
        # Return the original text if triple backticks are not present
        return text


df["cypher"] = [
    extract_text(el["body"]["choices"][0]["message"]["content"])
    for el in df["response"]
]

In [4]:
meta_df = pd.read_csv("text2cypher_questions.csv")
meta_df["id"] = [f"request-{el}" for el, _ in enumerate(meta_df["database"])]
meta_df.head()

,question,type,database,id
0,What are the top 5 products managed by the Hai...,Simple Retrieval Queries,neo4j,request-0
1,List the top 3 products with a molecule test t...,Simple Retrieval Queries,neo4j,request-1
2,Which 3 products have a pack price greater tha...,Simple Retrieval Queries,neo4j,request-2
3,Identify the top 5 countries where products wi...,Simple Retrieval Queries,neo4j,request-3
4,Which vendors supply the top 3 products with t...,Simple Retrieval Queries,neo4j,request-4


## Combining Batch output and Initial Q&A responses

In [5]:
combined_df = pd.merge(df, meta_df, left_on="custom_id", right_on="id")
combined_df = combined_df.sort_values(by="database")
combined_df.head()

,id_x,custom_id,response,error,cypher,question,type,database,id_y
0,batch_req_7b4DtF3MyMdqzud1PSUnnIKS,request-0,"{'status_code': 200, 'request_id': '61c2704410...",NaN,MATCH (o:Office{name: 'Haiti Field Office'})-[...,What are the top 5 products managed by the Hai...,Simple Retrieval Queries,neo4j,request-0
135,batch_req_HOASyGNA7P4Dlg5esmow1WqA,request-135,"{'status_code': 200, 'request_id': '3b2d8f61c7...",NaN,MATCH (p:Product)-[:GROUP]->(pg:Product_Group)...,List the product groups associated with produc...,Pathfinding Queries,neo4j,request-135
136,batch_req_Wharcr2wPzRrZF5fd68srsrE,request-136,"{'status_code': 200, 'request_id': 'fe0396d1f4...",NaN,MATCH (p:Product)-[:BRAND]->(b:Brand)\nWHERE p...,Show the first 3 brands associated with produc...,Pathfinding Queries,neo4j,request-136
137,batch_req_LhGyiq84K6JVbIOZ19mSz8Ig,request-137,"{'status_code': 200, 'request_id': '13714ab017...",NaN,MATCH (v:Vendor {name: 'ABBOTT LABORATORIES (P...,Which countries receive products from vendors ...,Pathfinding Queries,neo4j,request-137
138,batch_req_enCVun2gAOqRNCsu4USZcNlR,request-138,"{'status_code': 200, 'request_id': '503f2c2bcb...",NaN,MATCH (p:Product)-[:MANAGED_BY]->(:Office{name...,What are the names of products managed by the ...,Pathfinding Queries,neo4j,request-138


## Extracting all the Relevant Examples

So based on whether the query gets executed or not we classify each statement with Boolean operations

Only if we get many errors like `Neo.ClientError.Statement.TypeError` or `Neo.ClientError.Transaction.TransactionTimedOutClientConfiguration` we make sure such examples are not taken into consideration.


**Comparing the following parameters**

- Syntax Errors
- Timeout Errors
- Returning Any Results

In [6]:
syntax_error = []
returns_results = []
actual_errors = []
timeouts = []
last_graph = ""
for i, row in combined_df.reset_index().iterrows():
    if i % 100 == 0:
        print(i)
    # To avoid a new driver for every request
    if row["database"] != last_graph:
        last_graph = row["database"]
        print(last_graph)
        graph = Neo4jGraph(
            url=DEMO_URL,
            username=user,
            password=password,
            database=row["database"],
            refresh_schema=False,
            timeout=10,
        )
    try:
        data = graph.query(row["cypher"])
        if data:
            returns_results.append(True)
        else:
            returns_results.append(False)
        syntax_error.append(False)
        timeouts.append(False)
    except ValueError as e:
        if (("Generated Cypher Statement is not valid" in str(e)) | ("Neo.ClientError.Statement.TypeError" in str(e))):
            syntax_error.append(True)
            print(f"Syntax error in Cypher query: {e}")
        else:
            syntax_error.append(False)
            print(f"Other ValueError: {e}")
        returns_results.append(False)
        timeouts.append(False)
    except Exception as e:
        if (
            e.code
            == "Neo.ClientError.Transaction.TransactionTimedOutClientConfiguration"
        ):
            returns_results.append(False)
            syntax_error.append(False)
            timeouts.append(True)

    if len(syntax_error)!=(i+1):
        print("Problem at a certain point ----> ",str(i+1))
        #break
        returns_results.append(False)
        timeouts.append(False)
        syntax_error.append(False)
        actual_errors.append(i)


0
neo4j


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: product_group)} {position: line: 3, column: 8, offset: 62} for query: 'MATCH (p:Product)\nWHERE p.item_description IS NOT NULL\nWITH p.product_group AS productGroup, count(DISTINCT p.item_description) AS diversity\nRETURN productGroup, diversity\nORDER BY diversity DESC\nLIMIT 3'


Syntax error in Cypher query: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'NOT': expected
  "!="
  "%"
  "*"
  "+"
  ","
  "-"
  "/"
  "::"
  "<"
  "<="
  "<>"
  "="
  "=~"
  ">"
  ">="
  "AND"
  "CONTAINS"
  "ENDS"
  "IN"
  "IS"
  "OR"
  "STARTS"
  "XOR"
  "^"
  "}" (line 1, column 82 (offset: 81))
"MATCH (p:Product)-[:WEIGHT]->(:Country {name: 'Afghanistan', shipment_number: IS NOT NULL})"
                                                                                  ^}


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: sub_class)} {position: line: 4, column: 7, offset: 106} for query: "MATCH (p:Product)-[:WEIGHT]->(c:Country)\nWHERE p.pid IS NOT NULL\nAND p.item_description IS NOT NULL\nAND p.sub_class = 'Adult'\nRETURN DISTINCT c.name"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or tha

Problem at a certain point ---->  79


...

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: product_group)} {position: line: 4, column: 10, offset: 95} for query: "MATCH (p:Product)\nWHERE p.molecule_test_type = 'HIV'\nWITH p, COUNT(p) AS productCount\nRETURN p.product_group AS ProductGroup, productCount\nORDER BY productCount DESC\nLIMIT 3"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure

Syntax error in Cypher query: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: In a WITH/RETURN with DISTINCT or an aggregation, it is not possible to access variables declared before the WITH/RETURN: p (line 5, column 10 (offset: 126))
"ORDER BY p.pid"
          ^}
Syntax error in Cypher query: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: A pattern expression should only be used in order to test the existence of a pattern. It should therefore only be used in contexts that evaluate to a boolean, e.g. inside the function exists() or in a WHERE-clause. No other uses are allowed, instead they should be replaced by a pattern comprehension. (line 3, column 15 (offset: 68))
"WITH p, count((p)-[:GROUP]->(:Product_Group)) AS group_count"
               ^}


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: product_group)} {position: line: 3, column: 10, offset: 68} for query: "MATCH (p:Product)\nWHERE p.shipment_number STARTS WITH 'DN'\nRETURN p.product_group\nORDER BY p.pid\nLIMIT 3;"


200


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: sub_class)} {position: line: 2, column: 9, offset: 49} for query: "MATCH (p:Product)-[:WEIGHT]->(c:Country)\nWHERE p.sub_class = 'HIV test - Ancillary' AND p.shipment_number IS NOT NULL\nRETURN DISTINCT c.name"


In [8]:
combined_df["syntax_error"] = syntax_error
combined_df["timeout"] = timeouts
combined_df["returns_results"] = returns_results

## Extracting Relevant Columns

In [12]:
final_df = combined_df[
    [
        "question",
        "cypher",
        "type",
        "database",
        "syntax_error",
        "timeout",
        "returns_results",
    ]
]
final_df.head()

,question,cypher,type,database,syntax_error,timeout,returns_results
0,What are the top 5 products managed by the Hai...,MATCH (o:Office{name: 'Haiti Field Office'})-[...,Simple Retrieval Queries,neo4j,False,False,True
135,List the product groups associated with produc...,MATCH (p:Product)-[:GROUP]->(pg:Product_Group)...,Pathfinding Queries,neo4j,False,False,False
136,Show the first 3 brands associated with produc...,MATCH (p:Product)-[:BRAND]->(b:Brand)\nWHERE p...,Pathfinding Queries,neo4j,False,False,True
137,Which countries receive products from vendors ...,MATCH (v:Vendor {name: 'ABBOTT LABORATORIES (P...,Pathfinding Queries,neo4j,False,False,True
138,What are the names of products managed by the ...,MATCH (p:Product)-[:MANAGED_BY]->(:Office{name...,Pathfinding Queries,neo4j,False,False,False


In [13]:
# Assume df is your DataFrame and col1, col2, col3 are the boolean columns
distribution_col1 = final_df["syntax_error"].value_counts()
distribution_col2 = final_df["timeout"].value_counts()
distribution_col3 = final_df["returns_results"].value_counts()

print("Distribution for syntax_error:\n", distribution_col1)
print("Distribution for timeout:\n", distribution_col2)
print("Distribution for returns_results:\n", distribution_col3)

Distribution for syntax_error:
 syntax_error
False    207
True       5
Name: count, dtype: int64
Distribution for timeout:
 timeout
False    208
True       4
Name: count, dtype: int64
Distribution for returns_results:
 returns_results
True     131
False     81
Name: count, dtype: int64


## Final Output

This output file is built using GPT 3.5 and is used as the input for training our model

In [14]:
final_df.to_csv("text2cypher_gpt3.5turbo.csv", index=False)